In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import matplotlib 
import seaborn as sns
matplotlib.rcParams["figure.figsize"] = (20,10)

# Import data and process

In [ ]:
df=pd.read_csv('../input/anemia-dataset/anemia.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
features=df.columns
for feature in features:
     print(f'{feature}--->{df[feature].nunique()}')

In [ ]:
df.describe().T.sort_values(ascending =0,by='mean').style.background_gradient(cmap='BuGn').bar(subset=['std'], color='red').bar(subset=['mean'], color='blue')

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated()

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.duplicated()

In [ ]:
df.shape

In [ ]:
correl=df.corr()
fig = plt.figure(figsize=(15,7))
sns.heatmap(correl,annot=True)

In [ ]:
df.columns

In [ ]:
#continent_data.sort_values(by = '2022 Population',inplace = True)
plt.figure(figsize = (12,9))
features =['Gender', 'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Result']
features.reverse()# reverses the sorting order of the elements.
for feature in features:
    plt.plot(df[feature],label = feature)
plt.legend()
plt.show()

In [ ]:
pd.crosstab(df['Gender'], df['Result'])

In [ ]:
pd.crosstab(df['Gender'], df['Result']).plot(kind='bar')

In [ ]:
sns.countplot(x='Gender',data=df)
plt.show()

In [ ]:
#pd.crosstab(df['Gender'], df['Hemoglobin']).plot(kind='bar')


In [ ]:
sns.pairplot(df)

In [ ]:
plt.figure(figsize = (6,5))
sns.boxplot(x='Gender',y='Hemoglobin',data=df)
plt.show()
plt.figure(figsize = (6,5))
sns.boxplot(x='Gender',y='MCHC',data=df)
plt.show()

In [ ]:
df.columns

In [ ]:
sns.distplot(df['MCH'])
plt.show()

# the model

In [ ]:
X=df.drop(['Result'],axis='columns')

In [ ]:
y=df.Result

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
model_params = {
     'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
      'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
    
}

In [ ]:
scores = []
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
reg=RandomForestClassifier()
reg.fit(X_train, y_train)

In [ ]:
reg.score( X_test, y_test)

In [ ]:
y_predicted = reg.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
reg.predict(X_test)

In [ ]:
def predict_anim(Gender,Hemoglobin,MCH,MCHC,MCV):    
    gen_index = np.where(X.columns==Gender)[0]#
    x = np.zeros(len(X.columns))
    x[0] = Hemoglobin
    x[1] = MCH
    x[2] = MCHC
    x[3] = MCV
  
    if gen_index >= 0:
         x[gen_index] = 1

    return reg.predict([x])[0]

In [ ]:
predict_anim(1,5,10,48,21)

In [ ]:
import pickle
with open('anemia.pickle','wb') as f:
    pickle.dump(reg,f)